In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
import torch
from dotenv import load_dotenv
import io
from google.cloud import storage
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt

### This package will be included in our repo, but it is only a prompt template

In [4]:
from golemai.nlp.prompts import QUERY_INTRO_NO_ANS, SYSTEM_MSG_RAG, SYSTEM_MSG_RAG_SHORT
from golemai.nlp.llm_module import prepare_prompt
from golemai.nlp.llm_resp_gen import LLMRespGen

In [5]:
load_dotenv(), torch.cuda.is_available()

(True, True)

In [30]:
DATA_DIR = 'data'
DATASET_FILE = 'new_version_merged_df.parquet'
QUESTION_COL = 'question'
CONTEXT_COL = 'context'
ANSWER_COL = 'answer'

In [9]:
SCRIPTS_DIR = 'scripts'
GEMMA_DIR = 'gemma2_new_dataset'
ATT_DIR = 'attentions'

ATT_PATH = os.path.join('..', SCRIPTS_DIR, GEMMA_DIR, ATT_DIR)

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_path = os.path.join("..", DATA_DIR, DATASET_FILE)

### I will load our standard `unsloth/gemma-2-9b-it-bnb-4bit`

In [12]:
MODEL_ID = "unsloth/gemma-2-9b-it-bnb-4bit"
DTYPE = torch.bfloat16
LOAD_IN_4BIT = True

In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=os.environ["HF_TOKEN"])

In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map='auto',
    torch_dtype=DTYPE,
    token=os.environ["HF_TOKEN"],
    # sliding_window=8192
    # max_memory={0: "4GB", 1: "3GB"}
    #attn_implementation="sdpa",
)

In [9]:
reverted_tokenizer = {
    v: k
    for k, v in tokenizer.get_vocab().items()
}

In [71]:
sample_input_str = "Stolicą Polski jest Waszyngton DC bardzo lubię ten kraj."
problematic_span = [' Waszyngton DC']
toknized_input = tokenizer(sample_input_str, return_tensors="pt")

toknized_input.word_ids()

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [72]:
tokenized_span = tokenizer(problematic_span, return_tensors="pt", add_special_tokens=False)

In [73]:
for k in toknized_input['input_ids'][0]:
    print(f"{k}: {reverted_tokenizer[k.item()]}")

2: <bos>
5537: Sto
720: lic
235524: ą
70456: ▁Polski
9497: ▁jest
9626: ▁Was
1930: zy
512: ng
1166: ton
11399: ▁DC
38222: ▁bardzo
15747: ▁lub
44602: ię
2797: ▁ten
65170: ▁kraj
235265: .


In [74]:
for k in tokenized_span['input_ids'][0]:
    print(f"{k}: {reverted_tokenizer[k.item()]}")

9626: ▁Was
1930: zy
512: ng
1166: ton
11399: ▁DC


In [75]:
input_str_ids = toknized_input['input_ids'][0]
span_ids = tokenized_span['input_ids'][0]

In [76]:
input_str_ids, span_ids

(tensor([     2,   5537,    720, 235524,  70456,   9497,   9626,   1930,    512,
           1166,  11399,  38222,  15747,  44602,   2797,  65170, 235265]),
 tensor([ 9626,  1930,   512,  1166, 11399]))

In [77]:
def find_span_positions(input_ids, span_ids):
    
    input_len = len(input_ids)
    span_len = len(span_ids)
    
    for i in range(input_len - span_len + 1):

        if torch.equal(input_ids[i:i+span_len], span_ids):
            return i, i + span_len - 1
    
    return None, None

start_pos, end_pos = find_span_positions(input_str_ids, span_ids)
start_pos, end_pos

(6, 10)

In [78]:
labels_input = torch.zeros_like(input_str_ids)
labels_input[start_pos:end_pos + 1] = 1
labels_input

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [58]:
tokenized_input = tokenizer(sample_input_str, return_offsets_mapping=True)
tokenized_input

{'input_ids': [2, 5537, 720, 235524, 70456, 9497, 9626, 1930, 512, 1166, 11399, 38222, 15747, 44602, 2797, 65170, 235265], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 3), (3, 6), (6, 7), (7, 14), (14, 19), (19, 23), (23, 25), (25, 27), (27, 30), (30, 33), (33, 40), (40, 44), (44, 46), (46, 50), (50, 55), (55, 56)]}

### How does labels look like - small sample check

In [23]:
eval_files = [
    'gemma-2-9b-it-bnb-4bit_2024-11-17_13-44.json',
    'gemma-2-9b-it-bnb-4bit_2024-11-29_00-56.json',
    'gemma-2-9b-it-bnb-4bit_2024-11-29_01-31.json'
]

In [82]:
eval_df = pd.read_json(os.path.join('..', 'scripts', eval_files[0]), lines=True)

In [84]:
merged_df = None

for f in eval_files:

    eval_df = pd.read_json(os.path.join('..', 'scripts', f), lines=True)
    if 'index' in eval_df.columns:
        eval_df = eval_df.rename(columns={'index': 'id'})

    if merged_df is None:
        merged_df = eval_df
    else:

        merged_df = pd.concat([merged_df, eval_df], ignore_index=True).reset_index(drop=True)

In [123]:
merged_df

,id,document,ground_truth,response,decision,gpt4_explanation,problematic_spans,cost,prompt
0,0,#Document#: `Dokument [1]:` 18 sierpnia królow...,130,Wielka Armada liczyła 125 statków.,True,"The proposed answer states that ""Wielka Armada...",[],0.008440,You will be provided with a document and a pro...
1,1,"#Document#: `Dokument [ ""Post Malone"" ]:` 20 l...",Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,True,"The proposed answer states, ""Nie mogę udzielić...",[],0.005290,You will be provided with a document and a pro...
2,2,"#Document#: `Dokument [ ""Ouija (2015 film)"" ]:...",Ishita Dutta,Ishita Dutta,True,"The proposed answer states that ""Ishita Dutta""...",[],0.009995,You will be provided with a document and a pro...
3,3,"#Document#: `Dokument [ ""Gotham Independent Fi...",scenarzystka,Obydwóch reżyserzy filmowi.,False,The proposed answer states that both Sidney Lu...,"[""Obydwóch reżyserzy filmowi""]",0.006190,You will be provided with a document and a pro...
4,4,"#Document#: `Dokument [ ""Joanna Jędrzejczyk"" ]...",Nie mogę udzielić odpowiedzi na to pytanie na ...,W programie TVN Turbo MMAster.,False,The proposed answer states that Joanna Jędrzej...,"[""W programie TVN Turbo MMAster""]",0.004450,You will be provided with a document and a pro...
...,...,...,...,...,...,...,...,...,...
5992,poquad_v2_27489,"#Document#: `Dokument [ ""The Falling Man"" ]:` ...",106 i 107,Na 106 i 107 piętrze.,True,The document states that the restaurant Window...,[],0.003505,You will be provided with a document and a pro...
5993,poquad_v2_11468,"#Document#: `Dokument [ ""Red Hot Chili Peppers...",Give It Away,Give It Away,True,"The document states that the song ""Give It Awa...",[],0.003855,You will be provided with a document and a pro...
5994,poquad_v2_9258,"#Document#: `Dokument [ ""USS Wahoo (SS-238)"" ]...",tak,"Tak, w tekście wspomniano, że Komandor Kennedy...",True,"The proposed answer states that ""Komandor Kenn...",[],0.007040,You will be provided with a document and a pro...
5995,poquad_v2_8449,"#Document#: `Dokument [ ""Kalifornia"" ]:` Hiszp...",w wyniku powstania przeciwko władzy Hiszpanii,W wyniku powstania przeciwko władzy Hiszpanii ...,True,"The proposed answer states: ""W wyniku powstani...",[],0.004270,You will be provided with a document and a pro...


# Define Function to Find Indices
Define a function that takes a row of the DataFrame and finds the starting and ending indices of each substring in 'problematic_spans' within 'model_response'.

In [28]:
def find_indices(response, spans) -> list:
    """
    Function to find the starting and ending indices of each substring in 'problematic_spans' within 'model_response'.
    Returns a list of tuples (start, end) for each problematic span.
    If a span cannot be found, it returns (None, None).
    """

    indices = []
    if isinstance(spans, str):
        spans = [spans]

    if spans is None:
        return None
    
    for span in spans:
        start = response.rfind(span)
        if start != -1:
            end = start + len(span)
            indices.append((start, end))
        else:
            indices.append((None, None))
    
    return indices

In [66]:
df = pd.read_parquet(dataset_path)

In [41]:
df_res = pd.read_parquet(os.path.join('..', 'data', 'research_sample_with_indices.parquet'))

In [42]:
df_res.head(2)

,index,id,dataset,query,answer,context,formatted_context,context_length,answer_length,problematic_spans,model_response,gpt_index,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
0,4172,polqa_2503,polqa,Ile okrętów w 1588 r. liczyła Wielka Armada: 1...,130,`Dokument [1]:` 18 sierpnia królowa dokonała p...,<bos><start_of_turn>user\nYou are a helpful as...,1366,4,[],Wielka Armada liczyła 125 statków.,0,1123,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
1,1895,poquad_v2_4138,poquad_v2,"Po ilu dniach od premiery jego utwór ""Psycho"" ...",Nie mogę udzielić odpowiedzi na to pytanie na ...,"`Dokument [ ""Post Malone"" ]:` 20 lutego 2018 r...",<bos><start_of_turn>user\nYou are a helpful as...,607,17,[],Nie mogę udzielić odpowiedzi na to pytanie na ...,1,364,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False


In [131]:
merged_df.head(2)

,id,document,ground_truth,response,decision,gpt4_explanation,problematic_spans,cost,prompt,gpt_index,id_df_res
0,0,#Document#: `Dokument [1]:` 18 sierpnia królow...,130,Wielka Armada liczyła 125 statków.,True,"The proposed answer states that ""Wielka Armada...",[],0.00844,You will be provided with a document and a pro...,0.0,polqa_2503
1,1,"#Document#: `Dokument [ ""Post Malone"" ]:` 20 l...",Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,True,"The proposed answer states, ""Nie mogę udzielić...",[],0.00529,You will be provided with a document and a pro...,1.0,poquad_v2_4138


In [53]:
merged_df = merged_df.merge(df_res[['gpt_index', 'id']], left_on='id', right_on='gpt_index', how='left', suffixes=('', '_df_res'))
merged_df['id'] = merged_df['id_df_res'].combine_first(merged_df['id'])
merged_df.drop(columns=['id_df_res', 'gpt_index'], inplace=True)

In [ ]:
df.index = df['id']
merged_df.index = merged_df['id']

df = df.join(merged_df[['response', 'decision', 'problematic_spans']], how='inner')
df.drop(columns=['id'], inplace=True)
df.index.name = None

In [72]:

df = df.rename(columns={'response': 'model_response'})

In [73]:
df['problematic_spans'] = df['problematic_spans'].apply(lambda x: [span.removeprefix('"').removesuffix('"') for span in x] if x is not None else None)
df = df.loc[df['model_response'] != '<CUDA_ERROR>']

In [74]:
df.head(2)

,dataset,query,answer,context,context_length,model_response,decision,problematic_spans
bioask_1000,bioask,Czy możliwe jest oczyszczenie pseudopodiów do ...,Pseudopodia mogą być oczyszczane przy użyciu r...,`Dokument [1]:` Migracja komórek wymaga wypukł...,1547,"Tak, tekst mówi o metodzie oczyszczania pseudo...",True,[]
bioask_1001,bioask,Jaki jest mechanizm działania deaminazy cytydy...,Podczas odwrotnej transkrypcji APOBEC3G deamin...,`Dokument [1]:` Rodzina deaminaz deoksycytydyn...,2445,APOBEC3G hamuje replikację wirusa HIV-1 poprze...,True,[]


In [76]:
llm_rg = LLMRespGen(
    df=df,
    id_col='id',
    model_type='local',
    system_msg=SYSTEM_MSG_RAG_SHORT,
    prompt_template=QUERY_INTRO_NO_ANS,
    batch_size=1,
    device_num=1
)

llm_rg.tokenizer = tokenizer

Using device: cuda, device_num = 1


In [79]:
df['indices'] = df.apply(lambda row: find_indices(row['model_response'], row['problematic_spans']), axis=1)
df = df.loc[~df.index.isin([i for i, row in df.iterrows() if any(index == (None, None) for index in row['indices'])])]
df = df.drop(columns=['indices'])

In [ ]:
df

,dataset,query,answer,context,context_length,model_response,decision,problematic_spans
bioask_1000,bioask,Czy możliwe jest oczyszczenie pseudopodiów do ...,Pseudopodia mogą być oczyszczane przy użyciu r...,`Dokument [1]:` Migracja komórek wymaga wypukł...,1547,"Tak, tekst mówi o metodzie oczyszczania pseudo...",True,[]
bioask_1001,bioask,Jaki jest mechanizm działania deaminazy cytydy...,Podczas odwrotnej transkrypcji APOBEC3G deamin...,`Dokument [1]:` Rodzina deaminaz deoksycytydyn...,2445,APOBEC3G hamuje replikację wirusa HIV-1 poprze...,True,[]
bioask_1009,bioask,Ile selenoprotein jest zakodowanych w ludzkim ...,"25. 15kDa, DI1, DI2, DI3, GPx1, GPx2, GPx3, GP...",`Dokument [1]:` TŁO: Białka zawierające seleno...,1296,25,True,[]
bioask_101,bioask,Która z ludzkich selenoprotein zawiera kilka r...,"Selenoproteina P, która zawiera 10 selenocystein.",`Dokument [1]:` Selenoproteina P (Sepp1) jest ...,2282,Ludzka selenoproteina P (HSelP) zawiera 10 res...,True,[]
bioask_1021,bioask,Jak leczy się chorobę Riedela (zapalenie tarcz...,Zapalenie tarczycy typu Riedela jest rzadkim s...,`Dokument [1]:` Wstęp: Inwazyjne włókniste zap...,2457,Leczenie kortykosteroidami w zapaleniu tarczyc...,True,[]
...,...,...,...,...,...,...,...,...
poquad_v2_9744,poquad_v2,Co było przyczyną zniesienia święta Trzeciego ...,krwawe demonstracje studenckie,"`Dokument [ ""Święto Narodowe Trzeciego Maja"" ]...",347,Krwawych demonstracji studenckich w 1946 roku.,True,[]
poquad_v2_9909,poquad_v2,Dlaczego zamknięto pub Edwardsa w Dudley?,z powodu pożaru,"`Dokument [ ""Duncan Edwards"" ]:` Edwards uważa...",368,W 2001 jego imieniem nazwano jeden z pubów zna...,True,[]
poquad_v2_9939,poquad_v2,Jaki piłkarz uległ wypadkowi podczas Ligii Mis...,Nie mogę udzielić odpowiedzi na to pytanie na ...,"`Dokument [ ""Chelsea F.C."" ]:` Głównym celem M...",507,Nie mogę udzielić odpowiedzi na to pytanie na ...,True,[]
poquad_v2_9967,poquad_v2,"W którym roku prawdopodobnie wymyślono ""grę""?",1977,"`Dokument [ ""Gra (gra umysłowa)"" ]:` Geneza „G...",327,1977,True,[]


In [81]:
df

,dataset,query,answer,context,context_length,model_response,decision,problematic_spans
bioask_1000,bioask,Czy możliwe jest oczyszczenie pseudopodiów do ...,Pseudopodia mogą być oczyszczane przy użyciu r...,`Dokument [1]:` Migracja komórek wymaga wypukł...,1547,"Tak, tekst mówi o metodzie oczyszczania pseudo...",True,[]
bioask_1001,bioask,Jaki jest mechanizm działania deaminazy cytydy...,Podczas odwrotnej transkrypcji APOBEC3G deamin...,`Dokument [1]:` Rodzina deaminaz deoksycytydyn...,2445,APOBEC3G hamuje replikację wirusa HIV-1 poprze...,True,[]
bioask_1009,bioask,Ile selenoprotein jest zakodowanych w ludzkim ...,"25. 15kDa, DI1, DI2, DI3, GPx1, GPx2, GPx3, GP...",`Dokument [1]:` TŁO: Białka zawierające seleno...,1296,25,True,[]
bioask_101,bioask,Która z ludzkich selenoprotein zawiera kilka r...,"Selenoproteina P, która zawiera 10 selenocystein.",`Dokument [1]:` Selenoproteina P (Sepp1) jest ...,2282,Ludzka selenoproteina P (HSelP) zawiera 10 res...,True,[]
bioask_1021,bioask,Jak leczy się chorobę Riedela (zapalenie tarcz...,Zapalenie tarczycy typu Riedela jest rzadkim s...,`Dokument [1]:` Wstęp: Inwazyjne włókniste zap...,2457,Leczenie kortykosteroidami w zapaleniu tarczyc...,True,[]
...,...,...,...,...,...,...,...,...
poquad_v2_9744,poquad_v2,Co było przyczyną zniesienia święta Trzeciego ...,krwawe demonstracje studenckie,"`Dokument [ ""Święto Narodowe Trzeciego Maja"" ]...",347,Krwawych demonstracji studenckich w 1946 roku.,True,[]
poquad_v2_9909,poquad_v2,Dlaczego zamknięto pub Edwardsa w Dudley?,z powodu pożaru,"`Dokument [ ""Duncan Edwards"" ]:` Edwards uważa...",368,W 2001 jego imieniem nazwano jeden z pubów zna...,True,[]
poquad_v2_9939,poquad_v2,Jaki piłkarz uległ wypadkowi podczas Ligii Mis...,Nie mogę udzielić odpowiedzi na to pytanie na ...,"`Dokument [ ""Chelsea F.C."" ]:` Głównym celem M...",507,Nie mogę udzielić odpowiedzi na to pytanie na ...,True,[]
poquad_v2_9967,poquad_v2,"W którym roku prawdopodobnie wymyślono ""grę""?",1977,"`Dokument [ ""Gra (gra umysłowa)"" ]:` Geneza „G...",327,1977,True,[]


In [83]:
eval_df

,index,document,ground_truth,response,decision,gpt4_explanation,problematic_spans,cost,prompt
0,0,#Document#: `Dokument [1]:` 18 sierpnia królow...,130,Wielka Armada liczyła 125 statków.,True,"The proposed answer states that ""Wielka Armada...",[],0.008440,You will be provided with a document and a pro...
1,1,"#Document#: `Dokument [ ""Post Malone"" ]:` 20 l...",Nie mogę udzielić odpowiedzi na to pytanie na ...,Nie mogę udzielić odpowiedzi na to pytanie na ...,True,"The proposed answer states, ""Nie mogę udzielić...",[],0.005290,You will be provided with a document and a pro...
2,2,"#Document#: `Dokument [ ""Ouija (2015 film)"" ]:...",Ishita Dutta,Ishita Dutta,True,"The proposed answer states that ""Ishita Dutta""...",[],0.009995,You will be provided with a document and a pro...
3,3,"#Document#: `Dokument [ ""Gotham Independent Fi...",scenarzystka,Obydwóch reżyserzy filmowi.,False,The proposed answer states that both Sidney Lu...,"[""Obydwóch reżyserzy filmowi""]",0.006190,You will be provided with a document and a pro...
4,4,"#Document#: `Dokument [ ""Joanna Jędrzejczyk"" ]...",Nie mogę udzielić odpowiedzi na to pytanie na ...,W programie TVN Turbo MMAster.,False,The proposed answer states that Joanna Jędrzej...,"[""W programie TVN Turbo MMAster""]",0.004450,You will be provided with a document and a pro...
...,...,...,...,...,...,...,...,...,...
675,675,#Document#: `Dokument [1]:` TŁO: Pacjenci z za...,Celem badania klinicznego HAMLET (Hemicraniect...,<CUDA_ERROR>,True,The proposed answer states that the aim of the...,[],0.020580,You will be provided with a document and a pro...
676,676,"#Document#: `Dokument [ ""Schizofrenia"" ]:` Nie...",Nie mogę udzielić odpowiedzi na to pytanie na ...,<CUDA_ERROR>,True,The proposed answer states that it cannot prov...,[],0.005235,You will be provided with a document and a pro...
677,677,"#Document#: `Dokument [ ""Hunters Green"" ]:` Hu...",Niagara Falls,Niagara Falls,True,The proposed answer states that Martin Beck gr...,[],0.009395,You will be provided with a document and a pro...
678,678,"#Document#: `Dokument [ ""President of Guyana"" ...",Arthur Chung,Forbes Burnham,False,The proposed answer states that Forbes Burnham...,"[""Forbes Burnham""]",0.010650,You will be provided with a document and a pro...


In [86]:
for i, row in df.iterrows():
    print(f"Index: {i}")
    break   

Index: bioask_1000


In [99]:
foramtted_contexts = []
n_prompt_tokens = []
hallu_indicies = []
hallu_tokens = []

for i, row in df.iterrows():

    formatted_prompt = llm_rg._get_ready_prompt(
        row=row,
        prompt_columns=['query', 'context']
    )

    add_special_tokens = False if i in merged_df['id'].values else True

    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, add_special_tokens=add_special_tokens)
    n_prompt_tokens.append(inputs['input_ids'].shape[1])

    formatted_context = f"{tokenizer.decode(inputs['input_ids'][0])}{row['model_response']}"
    foramtted_contexts.append(formatted_context)

    hallu_idx = find_indices(formatted_context, row['problematic_spans'])
    hallu_indicies.append(hallu_idx)

    inputs = tokenizer(formatted_context, return_tensors="pt", padding=True, return_offsets_mapping=True, add_special_tokens=False)
    offset_mappings = inputs['offset_mapping'][0].tolist()

    hallu_mask = [0] * len(offset_mappings)

    for idx, (start, end) in enumerate(offset_mappings):

        for hallu_start, hallu_end in hallu_idx:
            if start >= hallu_start and end <= hallu_end:
                hallu_mask[idx] = 1
                
    hallu_tokens.append(hallu_mask)

df['formatted_context'] = foramtted_contexts
df['prompt_length'] = n_prompt_tokens
df['hallu_indicies'] = hallu_indicies
df['hallu_tokens'] = hallu_tokens
df['contain_hallu'] = df['problematic_spans'].apply(lambda x: True if x else False)

In [102]:
df.head(2)

,dataset,query,answer,context,context_length,model_response,decision,problematic_spans,formatted_context,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
bioask_1000,bioask,Czy możliwe jest oczyszczenie pseudopodiów do ...,Pseudopodia mogą być oczyszczane przy użyciu r...,`Dokument [1]:` Migracja komórek wymaga wypukł...,1547,"Tak, tekst mówi o metodzie oczyszczania pseudo...",True,[],<bos><start_of_turn>user\nYou are a helpful as...,1521,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
bioask_1001,bioask,Jaki jest mechanizm działania deaminazy cytydy...,Podczas odwrotnej transkrypcji APOBEC3G deamin...,`Dokument [1]:` Rodzina deaminaz deoksycytydyn...,2445,APOBEC3G hamuje replikację wirusa HIV-1 poprze...,True,[],<bos><start_of_turn>user\nYou are a helpful as...,2214,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False


In [103]:
df.to_parquet(os.path.join('..', DATA_DIR, 'all_examples_with_idx.parquet'))

In [134]:
for i, row in df.iterrows():

    if row['problematic_spans']:

        formatted_prompt = row['formatted_context']
        to_show_spans = [formatted_prompt[start:end] for start, end in row['hallu_indicies']]
        print(f"{row['index']}: {row['model_response'] = }\n{row['problematic_spans'] = }\n{to_show_spans = }\n")

3427: row['model_response'] = 'Obydwóch reżyserzy filmowi.'
row['problematic_spans'] = ['Obydwóch reżyserzy filmowi']
to_show_spans = ['Obydwóch reżyserzy filmowi']

2095: row['model_response'] = 'W programie TVN Turbo MMAster.'
row['problematic_spans'] = ['W programie TVN Turbo MMAster']
to_show_spans = ['W programie TVN Turbo MMAster']

2729: row['model_response'] = 'Komitet Organizacyjny miał złoto ze skradzionych przedmiotów po zagazowanych Żydach.'
row['problematic_spans'] = ['skradzionych przedmiotów']
to_show_spans = ['skradzionych przedmiotów']

4537: row['model_response'] = 'Nie mogę udzielić odpowiedzi na to pytanie na podstawie podanego kontekstu.'
row['problematic_spans'] = ['Nie mogę udzielić odpowiedzi na to pytanie na podstawie podanego kontekstu']
to_show_spans = ['Nie mogę udzielić odpowiedzi na to pytanie na podstawie podanego kontekstu']

4367: row['model_response'] = 'Crystal Springs is a ghost town in the Pahranagat Valley region of Lincoln County, Nevada in the Un

In [135]:
for i, h in zip(inputs['input_ids'][0], hallu_mask):
    print(f"{i}: {reverted_tokenizer[i.item()]}: {h}")

2: <bos>: 0
2: <bos>: 0
106: <start_of_turn>: 0
1645: user: 0
108: 
: 0
2045: You: 0
708: ▁are: 0
476: ▁a: 0
10055: ▁helpful: 0
20409: ▁assistant: 0
235265: .: 0
3883: ▁Your: 0
3356: ▁job: 0
877: ▁will: 0
614: ▁be: 0
577: ▁to: 0
3448: ▁answer: 0
3920: ▁questions: 0
31115: ▁accurately: 0
3482: ▁based: 0
611: ▁on: 0
573: ▁the: 0
2764: ▁given: 0
4807: ▁context: 0
578: ▁and: 0
780: ▁not: 0
861: ▁your: 0
8678: ▁internal: 0
5567: ▁knowledge: 0
235265: .: 0
108: 
: 0
141: ▁▁▁▁: 0
2495: If: 0
692: ▁you: 0
798: ▁can: 0
780: ▁not: 0
3448: ▁answer: 0
573: ▁the: 0
2872: ▁question: 0
1297: ▁only: 0
3482: ▁based: 0
611: ▁on: 0
573: ▁the: 0
4646: ▁provided: 0
4807: ▁context: 0
235269: ,: 0
2203: ▁return: 0
573: ▁the: 0
3448: ▁answer: 0
235292: :: 0
4103: ▁`: 0
28317: Nie: 0
153095: ▁mogę: 0
13869: ▁ud: 0
5139: zie: 0
133944: lić: 0
165557: ▁odpowiedzi: 0
1584: ▁na: 0
577: ▁to: 0
158462: ▁pytanie: 0
1584: ▁na: 0
145082: ▁podstawie: 0
3478: ▁pod: 0
112730: anego: 0
5224: ▁kon: 0
9837: tek: 0
16901: stu

### Main analysis - everything we got

In [105]:
df.head(2)

,dataset,query,answer,context,context_length,model_response,decision,problematic_spans,formatted_context,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
bioask_1000,bioask,Czy możliwe jest oczyszczenie pseudopodiów do ...,Pseudopodia mogą być oczyszczane przy użyciu r...,`Dokument [1]:` Migracja komórek wymaga wypukł...,1547,"Tak, tekst mówi o metodzie oczyszczania pseudo...",True,[],<bos><start_of_turn>user\nYou are a helpful as...,1521,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
bioask_1001,bioask,Jaki jest mechanizm działania deaminazy cytydy...,Podczas odwrotnej transkrypcji APOBEC3G deamin...,`Dokument [1]:` Rodzina deaminaz deoksycytydyn...,2445,APOBEC3G hamuje replikację wirusa HIV-1 poprze...,True,[],<bos><start_of_turn>user\nYou are a helpful as...,2214,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False


In [106]:
not_count, true_count = df['contain_hallu'].value_counts()

In [107]:
not_count, true_count

(5064, 832)

In [108]:
TEST_SIZE = 0.2

In [109]:
from sklearn.model_selection import train_test_split

In [110]:
train_set, test_set = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=42,
    stratify=df['contain_hallu']
)

In [111]:
train_set.to_parquet(os.path.join('..', 'data', 'train_all_sample.parquet'))
test_set.to_parquet(os.path.join('..', 'data', 'test_all_sample.parquet'))

### Filter only needed attension files

In [112]:
TRAIN_SAMPLE_FILE = 'train_all_sample.parquet'
TEST_SAMPLE_FILE = 'test_research_sample.parquet'

In [117]:
df = pd.read_parquet(os.path.join('..', DATA_DIR, TRAIN_SAMPLE_FILE))

In [118]:
df['dataset'].value_counts()

nq             1403
poquad_v2      1169
hotpotqa_en     847
polqa           560
hotpotqa_pl     520
bioask          217
Name: dataset, dtype: int64

In [120]:
df.head(2)

,dataset,query,answer,context,context_length,model_response,decision,problematic_spans,formatted_context,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
nq_507,nq,where is the capital city of alabama located,Montgomery,"`Dokument [ ""Alabama"" ]:` capital from 1817 to...",1598,Montgomery,True,[],<bos><start_of_turn>user\nYou are a helpful as...,1597,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
poquad_v2_27597,poquad_v2,Czy występowała kiedykolwiek w Polsce?,tak,"`Dokument [ ""Montserrat Caballé"" ]:` W 1963 od...",423,"Tak, tekst mówi, że ""Kilkakrotnie gościła takż...",True,[],<bos><start_of_turn>user\nYou are a helpful as...,422,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False


In [121]:
df['hallu_tokens']

nq_507                                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
poquad_v2_27597                         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
poquad_v2_45033                         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
hotpotqa_en_5ae3deb35542992e3233c488    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
poquad_v2_17864                         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                                              ...                        
hotpotqa_pl_5add2b435542990d50227e11    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
poquad_v2_11955                         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
polqa_6926                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
nq_223                                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
nq_679                                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: hall

In [122]:
OFFSET_SIZE = 16

In [11]:
def prep_att_pipe(
    att_path: str,
    n_context_tokens: int,
    n_prompt_tokens: int,
    offset_size: int = 16
) -> np.ndarray:
    """
    Function to prepare the attention tensor for further analysis.
    It removes the prompt tokens and the last offset_size tokens from the context.
    """

    att_tensor = np.load(att_path)
    print('Initial shape:', att_tensor.shape)
    att_tensor = att_tensor[..., n_prompt_tokens - offset_size: n_context_tokens, :n_context_tokens]
    print('Final shape:', att_tensor.shape)

    return att_tensor
        

In [15]:
from time import perf_counter

In [16]:
loading_times = []
tensor_sizes = []

In [19]:
for i, row in df.iterrows():

    start = perf_counter()

    att_file = f"{row['gpt_index']}.npy"
    print(f"att_file: {att_file}")

    n_context_tokens = len(row['hallu_tokens'])
    n_prompt_tokens = row['prompt_length']

    print(f"{n_context_tokens = }, {n_prompt_tokens = }")

    att_path = os.path.join(ATT_PATH, att_file)
    att_tensor = prep_att_pipe(att_path, n_context_tokens, n_prompt_tokens, OFFSET_SIZE)

    loading_times.append(perf_counter() - start)

    size_in_mb = att_tensor.nbytes / (1024 * 1024)
    tensor_sizes.append(size_in_mb)
    print(f"Size of att_tensor {row['gpt_index']}: {size_in_mb:.4f} MB")

    if i == 1:
        break

att_file: 340.npy
n_context_tokens = 335, n_prompt_tokens = 330
Initial shape: (42, 16, 338, 338)
Final shape: (42, 16, 21, 335)
Size of att_tensor 340: 9.0170 MB
att_file: 443.npy
n_context_tokens = 1674, n_prompt_tokens = 1665
Initial shape: (42, 16, 1677, 1677)
Final shape: (42, 16, 25, 1674)
Size of att_tensor 443: 53.6407 MB


In [21]:
att_tensor.shape

(42, 16, 25, 1674)

In [40]:
display(Markdown(f"### Mean loading time: {np.mean(loading_times):.4f} s"))
display(Markdown(f"### Mean tensor size: {np.mean(tensor_sizes):.4f} MB"))

### Mean loading time: 0.6715 s

### Mean tensor size: 44.3507 MB

In [41]:
N_SAMPLES = len(df)
display(Markdown(f"### Estimated time for loading all samples: {np.mean(loading_times) * N_SAMPLES:.2f} s"))
display(Markdown(f"### Estimated memory usage: {np.mean(tensor_sizes) * N_SAMPLES / 1024:.3f} GB"))

### Estimated time for loading all samples: 355.25 s

### Estimated memory usage: 22.912 GB

In [42]:
import psutil

available_memory = psutil.virtual_memory().available
available_memory_gb = available_memory / (1024 ** 3)

display(Markdown(f"### Available memory: {available_memory_gb:.3f} GB"))

### Available memory: 69.765 GB

In [43]:
from pprint import pprint

### Ok, so it is possible to load all data if we reshape / cut data in a smart way

In [44]:
df['dataset'].value_counts()

dataset
nq             142
poquad_v2      141
hotpotqa_en     89
polqa           67
hotpotqa_pl     53
bioask          37
Name: count, dtype: int64

In [45]:
df['contain_hallu'].value_counts()

contain_hallu
False    403
True     126
Name: count, dtype: int64

In [47]:
df_test = pd.read_parquet(os.path.join('..', 'data', 'new_version_merged_df.parquet'))

In [48]:
df_test

,id,dataset,question,answer,context
0,nq_0,nq,who got the first nobel prize in physics,Wilhelm Conrad Röntgen,"`Dokument [ ""Nobel Prize in Physics"" ]:` recei..."
1,nq_1,nq,when is the next deadpool movie being released,"May 18, 2018","`Dokument [ ""Deadpool (video game)"" ]:` 2015 i..."
2,nq_2,nq,the south west wind blows across nigeria between,till September,"`Dokument [ ""Geography of Nigeria"" ]:` south a..."
3,nq_3,nq,what does hp mean in war and order,hit points or health points,"`Dokument [ ""Order of War"" ]:` Order of War Or..."
4,nq_4,nq,who wrote the first declaration of human rights,Cyrus,"`Dokument [ ""Drafting of the Universal Declara..."
...,...,...,...,...,...
6219,bioask_440,bioask,Jaki jest odsetek osób reagujących na leczenie...,Tetrabenazyna jest stosowana empirycznie w lec...,`Dokument [1]:` WPROWADZENIE: Zwapnienia mózgu...
6220,bioask_715,bioask,Która kinaza jest hamowana przez Tripolinę A?,Tripolina A zmniejszała lokalizację pAurory A ...,`Dokument [1]:` Regulatory mitotyczne wykazują...
6221,bioask_1207,bioask,Jaka jest rola prognostyczna zmienionego profi...,Zmieniony profil tarczycy po operacji kardioch...,`Dokument [1]:` CEL: Pomimo poprawy postępowan...
6222,bioask_1327,bioask,Czy istnieje różnica w szybkości fuzji i rozsz...,"Tak. W kilku badaniach oszacowano, że fuzja i ...",`Dokument [1]:` Domeny białkowe są zwartymi ew...


In [47]:
df.head(3)

,index,id,dataset,query,answer,context,formatted_context,context_length,answer_length,problematic_spans,model_response,gpt_index,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
0,3265,poquad_v2_9553,poquad_v2,Jak nazywa się tradycyjne odzienie mieszkańców...,ta’ovala,"`Dokument [ ""Tonga na Zimowych Igrzyskach Olim...",<bos><start_of_turn>user\nYou are a helpful as...,573,4,[],`ta’ovala`,340,330,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
1,1601,nq_2342,nq,who hosted and won the inagural world cup,Uruguay,"`Dokument [ ""Patrick Nally"" ]:` marketing for ...",<bos><start_of_turn>user\nYou are a helpful as...,1908,2,[],Uruguay hosted and won the inaugural World Cup.,443,1665,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
2,1590,nq_2324,nq,who is president of india in present time,Ram Nath Kovind,"`Dokument [ ""A. P. J. Abdul Kalam"" ]:` 11th pr...",<bos><start_of_turn>user\nYou are a helpful as...,1719,5,[Nie mogę udzielić odpowiedzi na to pytanie na...,Nie mogę udzielić odpowiedzi na to pytanie na ...,218,1476,"[[6469, 6543]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True


### 
```python
Y.append(row['hallu_tokens'][n_prompt_tokens - OFFSET_SIZE: ])
```
### Here comes the trick - it is not the attenion vector for new token, but but previous one which **produced** that token. We need to find what characterizes the previous token attenions - they cause next token. And also attension pattern for last token is always the same so to describe that token we need to take its predestors. For example we can take the `window aproach` 

In [11]:
OFFSET_SIZE = 4
FIRST_N_TOKENS = 4

In [ ]:
from google.cloud import storage

client = storage.Client.from_service_account_json(
    os.path.join("..", "secrets", "storage-gcp-key.json")
)

In [23]:
for t in client.list_buckets():
    print(t)

<Bucket: hallucination-detection>


In [56]:
bucket_name = 'hallucination-detection'
bucket = client.bucket(bucket_name)

In [36]:
df.head(2)

,index,id,dataset,query,answer,context,formatted_context,context_length,answer_length,problematic_spans,model_response,gpt_index,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
0,3265,poquad_v2_9553,poquad_v2,Jak nazywa się tradycyjne odzienie mieszkańców...,ta’ovala,"`Dokument [ ""Tonga na Zimowych Igrzyskach Olim...",<bos><start_of_turn>user\nYou are a helpful as...,573,4,[],`ta’ovala`,340,330,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
1,1601,nq_2342,nq,who hosted and won the inagural world cup,Uruguay,"`Dokument [ ""Patrick Nally"" ]:` marketing for ...",<bos><start_of_turn>user\nYou are a helpful as...,1908,2,[],Uruguay hosted and won the inaugural World Cup.,443,1665,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False


In [45]:
df.loc[df['gpt_index'] == 351]

,index,id,dataset,query,answer,context,formatted_context,context_length,answer_length,problematic_spans,model_response,gpt_index,prompt_length,hallu_indicies,hallu_tokens,contain_hallu
15,2201,poquad_v2_7837,poquad_v2,Jak nazywał się ostatni krążownik liniowy na ś...,Nie mogę udzielić odpowiedzi na to pytanie na ...,"`Dokument [ ""Krążowniki liniowe typu Lion"" ]:`...",<bos><start_of_turn>user\nYou are a helpful as...,589,17,[],Nie mogę udzielić odpowiedzi na to pytanie na ...,351,346,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False


In [53]:
for i, row in df.iterrows():

    att_file = f"{row['gpt_index']}.npy"
    print(f"att_file: {att_file}")

    n_context_tokens = len(row['hallu_tokens'])
    n_prompt_tokens = row['prompt_length']

    print(f"{n_context_tokens = }, {n_prompt_tokens = }")

    att_path = os.path.join(ATT_PATH, att_file)
    att_tensor = prep_att_pipe(att_path, n_context_tokens, n_prompt_tokens, OFFSET_SIZE)

    buffer = io.BytesIO()
    np.save(buffer, att_tensor)
    buffer.seek(0)  

    blob_name = os.path.join("test_attensions", f"{row['gpt_index']}_reduced.npy")

    try:
        blob = bucket.blob(blob_name)
        blob.upload_from_file(buffer, content_type='application/octet-stream')
    except Exception as e:
        print(f"Error uploading {blob_name}: {e}")
    else:
        print(f'Array uploaded to gs://{bucket_name}/{blob_name}')
        
    print('\n\n')

att_file: 508.npy
n_context_tokens = 1662, n_prompt_tokens = 1626
Initial shape: (42, 16, 1665, 1665)
Final shape: (42, 16, 52, 1662)
Array uploaded to gs://hallucination-detection/test_attensions/508_reduced.npy



att_file: 506.npy
n_context_tokens = 365, n_prompt_tokens = 345
Initial shape: (42, 16, 368, 368)
Final shape: (42, 16, 36, 365)
Array uploaded to gs://hallucination-detection/test_attensions/506_reduced.npy



att_file: 361.npy
n_context_tokens = 1073, n_prompt_tokens = 1056
Initial shape: (42, 16, 1076, 1076)
Final shape: (42, 16, 33, 1073)
Array uploaded to gs://hallucination-detection/test_attensions/361_reduced.npy



att_file: 603.npy
n_context_tokens = 1525, n_prompt_tokens = 1499
Initial shape: (42, 16, 1528, 1528)
Final shape: (42, 16, 42, 1525)
Array uploaded to gs://hallucination-detection/test_attensions/603_reduced.npy



att_file: 170.npy
n_context_tokens = 1417, n_prompt_tokens = 1397
Initial shape: (42, 16, 1420, 1420)
Final shape: (42, 16, 36, 1417)
Array u

In [38]:
f_loaded_from_bucket = bucket.list_blobs()
for f in f_loaded_from_bucket:
    print(f.name)

attensions/
attensions/134_reduced.npy
attensions/143_reduced.npy
attensions/218_reduced.npy
attensions/219_reduced.npy
attensions/253_reduced.npy
attensions/256_reduced.npy
attensions/308_reduced.npy
attensions/317_reduced.npy
attensions/340_reduced.npy
attensions/351_reduced.npy
attensions/427_reduced.npy
attensions/443_reduced.npy
attensions/457_reduced.npy
attensions/469_reduced.npy
attensions/597_reduced.npy
attensions/657_reduced.npy


In [ ]:
X, Y = [], []

for i, row in df.iterrows():

    att_file = f"{row['gpt_index']}.npy"
    print(f"att_file: {att_file}")

    n_context_tokens = len(row['hallu_tokens'])
    n_prompt_tokens = row['prompt_length']

    Y.append(row['hallu_tokens'][n_prompt_tokens - OFFSET_SIZE: n_prompt_tokens + FIRST_N_TOKENS])

    print(f"{n_context_tokens = }, {n_prompt_tokens = }")

    att_path = os.path.join(ATT_PATH, att_file)
    att_tensor = prep_att_pipe(att_path, n_prompt_tokens + FIRST_N_TOKENS, n_prompt_tokens, OFFSET_SIZE)

    X.append(att_tensor)

    if i == 30:
        break

att_file: 340.npy
n_context_tokens = 335, n_prompt_tokens = 330
Initial shape: (42, 16, 338, 338)
Final shape: (42, 16, 8, 334)
att_file: 443.npy
n_context_tokens = 1674, n_prompt_tokens = 1665
Initial shape: (42, 16, 1677, 1677)
Final shape: (42, 16, 8, 1669)
att_file: 218.npy
n_context_tokens = 1493, n_prompt_tokens = 1476
Initial shape: (42, 16, 1496, 1496)
Final shape: (42, 16, 8, 1480)
att_file: 253.npy
n_context_tokens = 1492, n_prompt_tokens = 1469
Initial shape: (42, 16, 1495, 1495)
Final shape: (42, 16, 8, 1473)
att_file: 597.npy
n_context_tokens = 532, n_prompt_tokens = 528
Initial shape: (42, 16, 535, 535)
Final shape: (42, 16, 8, 532)
att_file: 457.npy
n_context_tokens = 1101, n_prompt_tokens = 1081
Initial shape: (42, 16, 1104, 1104)
Final shape: (42, 16, 8, 1085)
att_file: 657.npy
n_context_tokens = 971, n_prompt_tokens = 964
Initial shape: (42, 16, 974, 974)
Final shape: (42, 16, 8, 968)
att_file: 317.npy
n_context_tokens = 542, n_prompt_tokens = 537
Initial shape: (42, 

In [ ]:
bucket_name = 'hallucination-detection'
bucket = client.bucket(bucket_name)

blob_f = bucket.blob('train_attensions/0_reduced.npy').download_to_filename('0_reduced.npy')
att = np.load('0_reduced.npy')

In [68]:
for y in Y:
    print(y)

[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 1]
